# 演示使用向量数据库来检索。

https://www.xinpianchang.com/a12814093


https://haokan.baidu.com/v?pd=wisenatural&vid=5577726715250023515

https://cloud.tencent.com/developer/article/2328915

## 1.常用的向量数据库 faiss

### 1.1 生成测试数据集

In [2]:
import os
import socket
import socks

socks.set_default_proxy(socks.SOCKS5, "127.0.0.1", 10808)
socket.socket = socks.socksocket

In [5]:
from openai import OpenAI

client = OpenAI()

def generate_data_by_prompt(prompt):
    data = client.completions.create(
        model="gpt-3.5-turbo-instruct",
        prompt=prompt,
        max_tokens=2048,
        temperature=0.5,
        top_p = 1
    )
    return data.choices[0].text

prompt = """请你生成50条淘宝网里的商品的标题，每条在30个字左右，品类是3C数码产品，标题里往往也会有一些促销类的信息，每行一条。"""
data = generate_data_by_prompt(prompt)
print(data)



1. "超值特惠！华为P40 Pro 5G全网通手机，拍照更出色！"
2. "限时抢购！小米10 Pro 5G旗舰手机，性能强劲，价格更低！"
3. "升级换代！苹果MacBook Pro 16英寸笔记本电脑，轻薄便携，性能更强！"
4. "新品上市！华硕ROG游戏手机3，全新升级，让你畅玩游戏！"
5. "大容量存储！西部数据4TB移动硬盘，高速传输，轻松备份数据！"
6. "限量发售！索尼WH-1000XM4无线降噪耳机，带来沉浸式音乐体验！"
7. "超值套装！小米电视4A 65英寸+小米盒子S，享受家庭影院乐趣！"
8. "限时特惠！华为平板M6 10.8英寸，轻薄便携，性能出众！"
9. "新品推荐！联想小新Pro 13.3英寸笔记本电脑，轻薄便携，性能升级！"
10. "限时抢购！Apple Watch SE智能手表，多项功能，更便捷生活！"
11. "超值组合！华硕ROG游戏台式电脑+曲面显示器，畅玩游戏无压力！"
12. "限量发售！小米路由器AX3600，Wi-Fi6技术，更稳定的网络连接！"
13. "新品上市！华为MatePad Pro平板电脑，10.8英寸全面屏，轻松办公学习！"
14. "限时特惠！罗技G502 HERO游戏鼠标，高精度传感器，助你获胜！"
15. "超值套装！华硕灵耀X2 14英寸笔记本电脑+无线鼠标，轻松办公！"
16. "限时抢购！小米电视5 Pro 75英寸，8K超高清画质，带来震撼视觉体验！"
17. "升级换代！苹果iPhone 12 Pro Max，5G网络，拍照更出色！"
18. "新品推荐！华为MateBook X Pro 2020款，全新升级，轻薄便携！"
19. "限时特惠！索尼PS5游戏机，强大性能，畅玩游戏新体验！"
20. "超值组合！小米笔记本Pro 15.6英寸+小米无线鼠标，高效办公！"
21. "限量发售！联想拯救者Y9000X 15.6英寸游戏笔记本，性能强悍，畅玩游戏！"
22. "新品上市！华硕ZenFone 7 Pro 5G手机，旋转相机，拍照更灵活！"
23. "限时抢购！小米路由器AX1800，Wi-Fi6技术，更稳定的网络连接！"
24. "超值套装！华为平板M5 8.4英寸+蓝牙键盘，轻松办公学习！"
25. "限时特惠！罗技G915无线机械键盘，低延迟，提升游戏体验！

In [6]:
import pandas as pd

product_names = data.strip().split('\n')
df = pd.DataFrame({'product_name': product_names})
df.head()

,product_name
0,"1. ""超值特惠！华为P40 Pro 5G全网通手机，拍照更出色！"""
1,"2. ""限时抢购！小米10 Pro 5G旗舰手机，性能强劲，价格更低！"""
2,"3. ""升级换代！苹果MacBook Pro 16英寸笔记本电脑，轻薄便携，性能更强！"""
3,"4. ""新品上市！华硕ROG游戏手机3，全新升级，让你畅玩游戏！"""
4,"5. ""大容量存储！西部数据4TB移动硬盘，高速传输，轻松备份数据！"""


In [7]:
# 去掉序号
df.product_name = df.product_name.apply(lambda x: x.split('.')[1].strip())
df.head()

,product_name
0,"""超值特惠！华为P40 Pro 5G全网通手机，拍照更出色！"""
1,"""限时抢购！小米10 Pro 5G旗舰手机，性能强劲，价格更低！"""
2,"""升级换代！苹果MacBook Pro 16英寸笔记本电脑，轻薄便携，性能更强！"""
3,"""新品上市！华硕ROG游戏手机3，全新升级，让你畅玩游戏！"""
4,"""大容量存储！西部数据4TB移动硬盘，高速传输，轻松备份数据！"""


In [8]:
# 如法炮制，再生成一些女装的商品名称，覆盖不同的品类，这样后面我们演示搜索效果的时候就会方便一点。
clothes_prompt = """请你生成50条淘宝网里的商品的标题，每条在30个字左右，品类是女性的服饰箱包等等，标题里往往也会有一些促销类的信息，每行一条。"""
clothes_data = generate_data_by_prompt(clothes_prompt)
clothes_product_names = clothes_data.strip().split('\n')
clothes_df = pd.DataFrame({'product_name': clothes_product_names})
clothes_df.product_name = clothes_df.product_name.apply(lambda x: x.split('.')[1].strip())
clothes_df.head()

,product_name
0,夏日清凉必备！时尚女士短裙，轻松驾驭夏日时尚
1,限时特惠！百搭女士T恤，清凉一夏必备单品
2,轻松出游必备！时尚女士背带裤，舒适又百搭
3,夏日必备！清凉女士连衣裙，优雅迷人的夏日风情
4,限时抢购！高颜值女士手提包，轻松搭配时尚造型


In [9]:
df = pd.concat([df, clothes_df], axis=0)
df = df.reset_index(drop=True)
display(df)

,product_name
0,"""超值特惠！华为P40 Pro 5G全网通手机，拍照更出色！"""
1,"""限时抢购！小米10 Pro 5G旗舰手机，性能强劲，价格更低！"""
2,"""升级换代！苹果MacBook Pro 16英寸笔记本电脑，轻薄便携，性能更强！"""
3,"""新品上市！华硕ROG游戏手机3，全新升级，让你畅玩游戏！"""
4,"""大容量存储！西部数据4TB移动硬盘，高速传输，轻松备份数据！"""
...,...
95,夏日出街必备！时尚女士短裙，清凉又百搭的时尚装备
96,限时抢购！百搭女士休闲裤，舒适又时尚的夏日装备
97,惊喜促销！高颜值女士手提包，轻松搭配时尚造型
98,夏日度假必备！时尚女士防晒帽，时尚轻松穿出清凉感


### 1.2 将数据转换为embedding后进行存储

### 1.3 进行向量搜索